# Tutorial 3: MapReduce

## How to use this tutorial

  * Select "run all cells" on this notebook from the Run menu in Jupyter notebook or Jupyter
    lab. This step will produce intermediate data output and charts.
  * Some cells print out a url, which you can click on and bring up an interactive web UI to
    visualize the graph data.
  * In the unlikely event that the notebook becomes irresponsive, you can try "Restart
    Kernel" from the Kernel menu, then run individual cells one by one using `Shift+Enter`.
  * Some tutorials use local clusters consisting of multiple processes to mimic the effects
    of graph distribution over a remote cluster. By default, these local clusters
    automatically stop after idling for 15 minutes to conserve CPU and memory resources. You
    will need to rerun the entire notebook if your local cluster stopped due to inactivity.
  * Additional resources (video demos & blogs) are available at http://juliustech.co.
  * To report any issues, get help or request features, please raise an issue at
    https://github.com/JuliusTechCo/JuliusGraph/issues.

## 0. Introduction

In this tutorial, we use Julius RuleDSL to build a generic MapReduce pipeline, and
illustrate the benefits of Julius' high order rules.

[MapReduce](https://en.wikipedia.org/wiki/MapReduce) is a common pipeline pattern which is
often used for processing big data sets in parallel with multiple computers/workers. The
MapReduce pipeline is a defining feature in many popular data platforms, such as
[Hadoop](https://hadoop.apache.org/).

In this tutorial, we explain how to build a generic and re-usable MapReduce pipeline using a
few simple rules in Julius' low-code declarative RuleDSL, as oppose to writing tons of code
as in traditional programming languages.

The MapReduce pipeline is composed of three main steps:
- **map**: a common `map` function is applied to every input data batch.
- **shuffle**: workers redistribute the `map` output based on certain key values such that
  all data with the same key value is shipped to the same worker. After the shuffle, each
  worker has the complete mapping results for its sub set of keys.
- **reduce**: each worker then process all the mapping results for a subset of keys by
  applying a `reduce` function. These reduced results are then collated as the final result.

The following image shows the generic MapReduce steps for the problem of counting the number
of occurrences of each word in a large collection of documents, which is given in the original
paper of Hadoop. This word count example is often used to illustrate the MapReduce pipeline.
We will try to replicate this example while building a generic MapReduce pipeline in Julius
RuleDSL from scratch.

![](../assets/mapreduce.png)

The MapReduce input data is a collection of data batches. The creation of the data batches
has to be done before the MapReduce pipeline (as in the Splitting stage in the diagram
above). For example, if the original data is a single large data file, it has to be split
into multiple batches of smaller files before feeding into the MapReduce process.

The goal of the tutorial is to construct a generic `mapreduce` rule whose `mapper`,
`shuffler` and `reducer` operators can be customized by the user. Even though the word
count problem itself is trivial, we will implement the pipeline in a generic fashion so that
it can be re-used without change for any MapReduce problems.

The readers are referred to the quick start tutorial for basic concepts and syntax of
`RuleDSL` and `Atom`. But for completeness, we give a brief explanations of the rule syntax
and graph execution here.

A rule in `RuleDSL` has the following syntax:

```julia
RuleDSL.@addrules namespace begin
    rulename(rulearg1::Type1, rulearg2::Type2, ...) = begin
        # additional code here transforming ruleargs to atom args and dependent args
        AtomName[atomarg1, atomarg2...](deprule1(depargs1...), deprule2(depargs2...), ...)
    end
end
```

The `RuleDSL.@addrules` is a macro used for processing RuleDSL, it takes a namespace
parameter and a set of rule declarations.  The rule namespace helps organize the rules into
related groups, and avoid name clashes. A rule in RuleDSL is an instruction to create
certain nodes in the computational graph. When Julius GraphEngine process a rule, it creates
a node with name `rulename`in the graph. The `AtomName[atomarg1, atomarg2...]` syntax
defines an `Atom` object, which is used to process the data from the node's dependency.
The dependent nodes are captured by the `deprules1, deprules` etc, they are added
recursively to the graph, if they do not already exist.

As you can appreciate, graph programming is quite different from traditional programming.
Instead of writing imperative functions, we declare the logic and dependencies using rules,
then let the Graph Engine to create the application or systems as computational DAGs for us.
The benefit of graph programming will become more aparent when we go through this tutorial.

The generic `mapreduce` rule should include three stages, mapper, shuffler and reducer,
therefore it should look like:

```julia
RuleDSL.@addrules mr begin
    mapreduce(
        batches::Vector{RuleDSL.NodeRef},
        mapper::RuleDSL.NodeRef,
        shuffler::RuleDSL.NodeRef,
        reducer::RuleDSL.NodeRef
    ) = begin
        # ... rule definition goes here ...
    end
end
```

The `RuleDSL.NodeRef` is a data structure that refers to another node in the graph. In Julius,
every node is created by a specific rule, therefore a dependency on another node can also be
understood as a dependency on another rule in the `RuleDSL`. A rule with a
`RuleDSL.NodeRef` parameter, like the `mapreduce` rule above, is called a high order rule,
as it defines a generic pattern whose behavior depends on other nodes (or rules). Each
`RuleDSL.NodeRef` parameter can be a node/rule that has a complex sub-graph, thus the high
order rule is extremely powerful in defining complex logic and behaviors.
Furthermore, a high order rule can be passed as parameter to another rule, creating even
higher order rules. The ability to nest high order rules is one of the reasons why the
`RuleDSL` is both low-code and expressive. The high order rule is similar in
spirit to the high order functions in functional programming, which we will compare to at
the end of this tutorial.

We now proceed to the implementation of the MapReduce pipeline as depicted in the diagram
above using Julius RuleDSL.

## 2. Generic Map/Reduce

### 2.1 Mapping

In the mapping step of the word count example, a batch of data is just a `String` such as
`"Mary has a lamb"`, which is converted into a `Vector` of `Pairs`: `["Mary" => 1, "has" =>
1, "a" => 1, "lamb" => 1]`, where each entry represents one occurrence of a given word. At
the shuffle stage, the vector is split by a key value, which is the word itself in the above
diagram; then all the pairs for the same key word are sent to a single node; where they are
concatenated to form a single vector. Finally at the reducer stage, the total occurrence of
each key word is deduced by simply counting their occurrences in the vector.

Given that the logic in the word count example is simple, we use a generic `ApplyFn` atom
that is provided as part of the `DataScience` package, which can take any Julia function as
argument, so that we don't have to define many Atom types for every stage of the
MapReduce process. ApplyFn source code is listed below, which inherits from the abstract
base type `Datom` and implements a generic method `fwddata!`, which will be called by Julius
Graph Engine at run time to process data at individual node.

```Julia
import GraphEngine.RuleDSL: fwddata!

struct ApplyFn <: RuleDSL.Datom
    fn::Any # can be a function or any object with a callable method defined
    params::Tuple # the first few arguments of `fn`

    ## this inner constructor captures `params` as a Tuple
    ApplyFn(fn::Any, params::Any...) = new(fn, params)
end

fwddata!(self::ApplyFn, xs::Any...) = [self.fn(self.params..., xs...)]
```

Using the generic `DataScience.ApplyFn` Atom, the `mapper` rule for word count example can
be written as:

In [ ]:
using GraphEngine: RuleDSL, GraphVM
using DataScience: ApplyFn
using AtomExt

wordmap(words::String) = [k => 1 for k in split(words)]

RuleDSL.@addrules mr begin
    mapper(batch::RuleDSL.NodeRef, mapfun::Function) = ApplyFn[mapfun](batch...)
end

The `GraphEngine.RuleDSL` and `GraphEngine.GraphVM` modules has to be included in order to
use the RuleDSL to create and run computational graphs.

The dependency of this rule is simply given as `batch...`, which specifies that the node
represented by the `batch` parameter is a dependency. The three dot syntax `...` is used to
signal dynamic dependencies from a `NodeRef` parameter or variable. At run time, Julius
GraphEngine first converts the `ApplyFn[mapfun]` specification to a call to the
constructor of `ApplyAtom(mapfun)`. Then, the generic `fwddata!` method of the `ApplyFn`
atom object is used to process the data from its input node specified by the `batch`
parameter.

The `mapper` rule above takes a single `RuleDSL.NodeRef` as argument, as it only applies to an
individual batch. However, the `mapreduce` rule needs to process all the mapper
results from all batches. So, how do we make that information available to the `mapreduce`
rule? We could create a collection of mapper rules as `Vector{NodeRef}` then pass it into
the `mapreduce` rule:

```julia
mappers = RuleDSL.@ref(mr.mapper(batch, mapfun) for batch in batches)
mr = RuleDSL.@ref mr.mapreduce(batches, mappers, shufflers, reducers)
```

where the `batches` is a `Vector{RuleDSL.NodeRef}` representing the collection of input
batches. However, this approach would require us to also create vectors of `shufflers` and
`reducers`, thus putting too much burden on the user to ensure their consistencies. By
observing that the first argument of the `mapper` rule is its input data batch, and the same
mapper rule should be applied to all batch inputs, we instead choose to drop the first
argument in the mapper rule before passing it as an argument to the `mapreduce` rule, such
that:

```julia
mapper = RuleDSL.@ref mr.mapper(mapfun)
mr = RuleDSL.@ref mr.mapreduce(batches, mapper, shuffler, reducer)
```

Inside the `mapreduce` rule, the first argument is added back for every data batch using the
following `prepend` function, to recover the full form of the `mapper` rule:

In [ ]:
prepend(ref::RuleDSL.NodeRef, firstarg::Any) = RuleDSL.NodeRef(ref.ns, ref.name, (firstarg, ref.params...), ref.meta)

The advantages of dynamically inserting the first parameter in the `mapreduce` rule are the
following:
- First it is more readable and clear in that we only need the overall rule logic, but not
  its first argument that specifies its batch input.
- Secondly it is less error prone, as the mappers are created inside the `mapreduce` rule to
  be fully consistent with the batches. We will do the same for `shuffler` and `reducer`
  later.

Let's test our mapping rule to see how it works. We have to define the input data batches
first. For this word count example, we can simply use the `ApplyFn` atom with the
`identity` function to return a rule argument, such that:

In [ ]:
@addrules mr begin
    batch(s::Any) = ApplyFn[identity, s]()
end

# some input data
_sentences = ("Deer Bear River", "Car Car River", "Deer Car Bear")
_batches   = RuleDSL.@ref(mr.batch(s) for s in _sentences)
_mapper    = RuleDSL.@ref mr.mapper(wordmap)

# prepend returns a new `NodeRef` such that `mappers` is of `Vector{NodeRef}` type
_mappers = [prepend(_mapper, batch) for batch in _batches]

# create a local graph, provide the node references and calculate
config = RuleDSL.Config()
gs1 = GraphVM.createlocalgraph(config, RuleDSL.GenericData())
GraphVM.calcfwd!(gs1, Set(_mappers))

We have now created a computational graph for the mapper and executed it. How do we see the
results? Julius provides an easy-to-use web UI for users to navigate and visualize the
resulting data and logic in the graph. The following code block starts a local server so
that the web UI can retrieve the resulting graph data, it also overrides the
`RuleDSL.nodelabel` method to customize the information displayed on the graph node.

In [ ]:
using GraphIO

# a container of graphs
gss = Dict{String,RuleDSL.AbstractGraphState}()

# used for WebUI display purposes
port = GraphVM.drawdataport()
@async GraphVM.startresponder(gss, port);

# override node label display
import GraphEngine.RuleDSL: nodelabel
function nodelabel(gs::RuleDSL.AbstractGraphState, ref::RuleDSL.NodeRef)
    shortrepr(x::Vector; sep=", ") = "["*join(shortrepr.(x), sep)*"]"
    shortrepr(p::Pair; sep="=>") = shortrepr(p.first) * sep * shortrepr(p.second)
    shortrepr(p::Dict; sep=", ") = "{" * join(shortrepr.(collect(p)), sep) * "}"
    shortrepr(x::Any; sep="") = repr(x)

    label = haskey(ref.meta, :label) ? ref.meta[:label] : "$(ref.ns).$(ref.name)"

    try
        data = RuleDSL.getdata(gs, ref)
        if isone(length(data))
            data = first(data)
        end
        label *= "\n" * shortrepr(data; sep = "\n")
    catch
        label *= ": n/a"
    end

    return label
end

User can view the resulting data from exeucting the graph interactively by clicking on the
url below to bring up the full web UI. As expected, the output of mapper is a vector of
entries like `"word" => 1`.

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs1, port; key="map");
display("image/svg+xml", svg)

### 2.2 Shuffling

The shuffling step consists of three sub steps:
1. take the outputs from the `mappers` and split them into multiple chunks by certain key
   values computed from the mapped data.
2. move these chunks around so that all data with the same key value is shipped to the same
   node.
3. concatenate all the chunks associated with a given key at the containing node to recover
   the full collection of data for the key.

To implement the first sub step of the shuffling, we define a generic split function that
takes a key function:

In [ ]:
# given a collection of elements `xs` and a key function that computes the key of each of
# these elements, return a Dictionary of `key => x`
function splitbykey(keyfunc::Function, xs::Any)
    splits = Dict()
    for x in xs
        key = keyfunc(x)
        splits[key] = push!(get(splits, key, []), x)
    end
    return splits
end

With this split function, we define three rules that corresponds to the three sub steps of
shuffling, then combine them together in the generic shuffler rule:

In [ ]:
@addrules mr begin

    # use `splitbykey` function defined above
    splitbykey(mapper::RuleDSL.NodeRef, keyfunc::Function) = ApplyFn[splitbykey, keyfunc](mapper...)

    # select an element of a dictionary ir exists or return an empty `Vector{Any}`
    selectkey(dict::RuleDSL.NodeRef, key::Any; label="selectby $(key)") = ApplyFn[dict -> get(dict, key, [])](dict...)

    # merge
    mergebykey(vecs::Vector{RuleDSL.NodeRef}) = ApplyFn[vcat](vecs...)


    shuffler(mappers::Vector{RuleDSL.NodeRef}, keyfunc::Function, keys::Set) = begin

        splits = RuleDSL.@ref(splitbykey(mapper, keyfunc) for mapper in mappers)

        shuffled = Vector{NodeRef}()
        for key in keys
            # a `Vector{NodeRef}` that encompasses nodes with a given key
            selected = RuleDSL.@ref(selectkey(s, key) for s in splits)

            # merge the previously selected nodes outputs
            merged = RuleDSL.@ref mergebykey(selected; label="mergeby $key")

            # add merged element to the shuffled `Vector`
            push!(shuffled, merged)
        end

        Alias(shuffled...)
    end
end

These rules are quite self explanatory as all of them use the generic atom `ApplyFn`.
It is worth mentioning that the `selectkey` rule uses a function closure when constructing
the `ApplyFn` atom; and in the `mergebykey` rule, the  `...` follows a `Vector{NodeRef}`
to specify dynamic dependency on multiple rules in the vector. The `label` keyword in the
 `selectkey` rule is to customize the display information of individual nodes in the graph
 web UI, please refer to the `nodelabel` function defined at the beginning.

We can test the shuffler using the words in the text as the split key, the `first` in the
shuffler rule is a function that returns the first element of the `"word"=>1` pair, which is
the word itself.

In [ ]:
# _mappers were created before
_shuffler = RuleDSL.@ref mr.shuffler(_mappers, first, Set(["Bear", "Car", "Deer", "River"]))

gs2 = GraphVM.createlocalgraph(config, RuleDSL.GenericData())
GraphVM.calcfwd!(gs2, Set([_shuffler]))

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs2, port; key="mappers");
display("image/svg+xml", svg)

### 2.3 Reducing

Finally, we get to the *reduce* part of the MapReduce pipeline. In the word count example,
the `reducer` simply counts the occurrences of word. The `reducer` rule is applied to the
result of `mergebykey`, i.e. a vector of entries like `"word" => 1`, which are all
associated with the same key value from shuffling stage.

In [ ]:
RuleDSL.@addrules mr begin
    reducer(shuffled::RuleDSL.NodeRef, reducefun::Function) = ApplyFn[reducefun](shuffled...)
end

# the reducer function
function wordreduce(xs::Vector)
    count = Dict()
    for (key, _) in xs
        count[key] = get(count, key, 0) + 1
    end
    return count
end

### 2.4 Map/Reduce Rule

We now put everything together and write a generic `mapreduce` rule. Note that we will use
the same `prepend` function to dynamically insert the first argument the for `shuffler` and
`mapper` rules:

In [ ]:
RuleDSL.@addrules mr begin
    mapreduce(
        batches::Vector{RuleDSL.NodeRef},
        mapper::RuleDSL.NodeRef,
        shuffler::RuleDSL.NodeRef,
        reducer::RuleDSL.NodeRef
    ) = begin

        # create one mapper node per batch
        mappers = [prepend(mapper, batch) for batch in batches]

        # create the shuffler
        shuffler = prepend(shuffler, mappers)

        # this gives the inputs to the shuffled nodes, which is where reducer must be applied
        shuffled = RuleDSL.calcdeps(RuleDSL.@config, shuffler)
        reducers = [prepend(reducer, m) for m in shuffled]

        # finally the results (i.e. a Dict per reducer) are merged to a single Dictionary
        ApplyFn[merge](reducers...)
    end
end

Let's test the MapReduce rule using our word count example:

In [ ]:
# no need for the first argument as it will be populated at `mapreduce`
_shuffler = RuleDSL.@ref mr.shuffler(first, Set(["Bear", "Car", "Deer", "River"]))

_mapper  = RuleDSL.@ref mr.mapper(wordmap)
_reducer = RuleDSL.@ref mr.reducer(wordreduce)

_mapreduce = RuleDSL.@ref mr.mapreduce(_batches, _mapper, _shuffler, _reducer)

gs3 = GraphVM.createlocalgraph(config, RuleDSL.GenericData())
GraphVM.calcfwd!(gs3, Set([_mapreduce]))

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs3, port; key="mapred");
display("image/svg+xml", svg)

The resulting diagram from Julius web UI is quite self explanatory, it matches exactly the
diagram provided by the Hadoop paper. A side benefit of Julius is that it frees developers
from the pain of having to manually draw system diagram or UMLs ever again. The graph
diagram above is an output from the Julius Graph Engine, which shows great details of both
the data and logic. Julius' convenient Web UI allows users to easily navigate and access the
entire graph data and logic, which can be accessed by clicking the link above if you are
running this example in Jupyter.

### 2.5 Split by Hashed Keys

So far, our MapReduce implementation works as expected. However, there is a serious
shortcoming in that we have to specify all the possible words in the shuffler, which is not
known before we process all the input batches. In practice, we don't want to scan all the
input batches just to find out all the possible words, which can be very time consuming when
the inputs are large. Also, in live streaming applications, such pre-scan is not possible at
all.

It would be much more convenient if we don't have to specify all the possible words in the
shuffler. We can easily achieve this by supplying a different key function whose number of
possible outputs are known, for example, by making use of the `hash` and the remainder `%`
functions:

In [ ]:
_shuffler = RuleDSL.@ref mr.shuffler(x -> Int(hash(first(x)) % 3), Set(collect(0:2)))

# reuse the same _mapper and _reducer declared earlier
_mapreduce = RuleDSL.@ref mr.mapreduce(_batches, _mapper, _shuffler, _reducer)

gs4 = GraphVM.createlocalgraph(config, RuleDSL.GenericData())
GraphVM.calcfwd!(gs4, Set([_mapreduce]))

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs4, port; key="hash");
display("image/svg+xml", svg)

Now the shuffler splits the mapper data into 3 pipes, each of which is identified by
an index number. In this implementation, multiple words can go to the same pipe. This
implementation removes the need of pre-scans for obtaining all the words; it also works for
live streaming use cases. Since the splitting by hash key is a much better implementation,
we declare a couple convenience rules to encourage its use:

In [ ]:
@addrules mr begin

    splitbykey(mapper::RuleDSL.NodeRef, keyfunc::Function, N::Int) = begin
        ApplyFn[splitbykey, x -> Int(hash(keyfunc(x)) % N)](mapper...)
    end

    shuffler(mappers::Vector{RuleDSL.NodeRef}, keyfunc::Function, N::Int) = begin

        splits = RuleDSL.@ref(splitbykey(mapper, keyfunc, N) for mapper in mappers)

        shuffled = Vector{NodeRef}()
        for key in 0:N-1

            # a `Vector{NodeRef}` that encompasses nodes with a given key
            selected = RuleDSL.@ref(selectkey(s, key) for s in splits)

            # merge the previously selected nodes outputs
            merged = RuleDSL.@ref mergebykey(selected; label="mergeby $key")

            # add merged element to the shuffled `Vector`
            push!(shuffled, merged)
        end

        Alias(shuffled...)
    end
end

Now, the shuffler declaration can be simply given as:

In [ ]:
_shuffler = RuleDSL.@ref mr.shuffler(first, 3)

which becomes much easier to read and define than its equivalent earlier version of
`_shuffler`. Note that since rules support polymorphism, the hash version of `splitbykey`
rule will be used if an interger is supplied as its 3rd argument.

So far we have demonstrated the MapReduce pipeline can be implemented using RuleDSL by
simply declaring a few high order rules. The resulting MapReduce rule is generic, powerful
and reusable. Next, we will use it to solve a few common MapReduce problems.

## 3. Examples of MapReduce

### 3.1 Finding Friends

We can use the MapReduce pipeline to compute the common friends among hundred of millions or
more users in a social network. This feature can be used to populate the *You and Joe have N
friends in common* displayed in many social networks. Given the list of friends for each
user, we proceed to define both a `mapper` and a `reducer` functions and make use of our
previously defined `mapreduce` rule to compute common friends for every user pair $\left(
u_i, u_j \right)$:

In [ ]:
function friends_mapfun(batch::String)
    dict = Dict{NTuple{2,Char},Vector}()
    handler = strip.(split(batch, "=>"))

    # no friends
    if isone(length(handler))
        return [dict]
    elseif length(handler) > 2
        return error("Unexpected data format.")
    end

    user, friends = handler

    # no friends
    if isempty(friends)
        return dict
    end

    uid = only(user)
    fids = only.(split(friends, ','))
    for fid in fids
        if isequal(uid, fid)
            continue
        end

        key = tuple(sort!([uid, fid])...)
        push!(dict, key => fids)
    end

    return dict
end

function friends_reducefun(shuffler::Vector)
    out = Dict{NTuple{2,Char},Vector{Char}}()
    for (k, v) in shuffler
        if !haskey(out, k)
            out[k] = v
        else
            out[k] = intersect(out[k], v)
        end
    end
    return out
end

# each user is represented by a `Char`
_friends = IOBuffer("
    A => B,C,D
    B => A,C,D,E
    C => A,B,D,E
    D => A,B,C,E
    E => B,C,D
")

_batches = RuleDSL.@ref(mr.batch(line) for line in eachline(_friends) if !isempty(line))

_mapreduce = RuleDSL.@ref mr.mapreduce(
    _batches,
    RuleDSL.@ref(mr.mapper(friends_mapfun)),
    RuleDSL.@ref(mr.shuffler(first, 4)),
    RuleDSL.@ref(mr.reducer(friends_reducefun))
)

In [ ]:
gs5 = GraphVM.createlocalgraph(config, RuleDSL.GenericData())
GraphVM.calcfwd!(gs5, Set([_mapreduce]))

In [ ]:
svg = GraphIO.postlocalgraph(gss, gs5, port; key="ff");
display("image/svg+xml", svg)

### 3.2 GroupBy

When dealing with large data set, we often need to split it to smaller batches, then apply
the MapReduce pipeline to perform certain operations on individual batches then group
them together. In this section, we will show how to implement the `groupby` operation
on a large data set using the MapReduce pipeline.

In order to split the data in multiple batches, we make use of our `DDataFrame` (which stands
for Distributed DataFrames) provided in the `DataScience` package. The following `mapper`
and `reducer` rules implements the group by using any number of features within the
`MapReduce` pipeline:

In [ ]:
using DataFrames
using DataScience: DDataFrame

# `cols` can be anything accepted by `DataFrames.groupby` method
function groupby_mapfun(batch::AbstractDataFrame, cols)
    dict = Dict()
    gdf = groupby(batch, cols)
    for (key, df) in zip(keys(gdf), gdf)
        push!(dict, NamedTuple(key) => DataFrame(df; copycols=false))
    end
    return dict
end

function groupby_reducefun(shuffler::Vector)
    out = Dict()
    for (k, v) in shuffler
        out[k] = append!(get(out, k, DataFrame()), v)
    end
    return out
end

filepath = joinpath(@__DIR__, "../data/iris.csv")
ddf = DDataFrame(filepath, nrows=25)
_batches = ddf.chunks

# use 3 reducing nodes for the reducing step
_mapreduce = RuleDSL.@ref mr.mapreduce(
    _batches,
    RuleDSL.@ref(mr.mapper(x -> groupby_mapfun(x, [:Species]))),
    RuleDSL.@ref(mr.shuffler(first, 3)),
    RuleDSL.@ref(mr.reducer(groupby_reducefun))
)

In [ ]:
gs6 = GraphVM.createlocalgraph(config, RuleDSL.GenericData())
GraphVM.calcfwd!(gs6, Set([_mapreduce]))

In [ ]:
nodelabel(::AbstractGraphState, ref::NodeRef) = haskey(ref.meta, :label) ? ref.meta[:label] : "$(ref.ns).$(ref.name)"
svg = GraphIO.postlocalgraph(gss, gs6, port; key="groupby");
display("image/svg+xml", svg)

The result is a `DataFrame` per group, such that, the first 10 rows look like:

In [ ]:
_reducers = calcdeps(config, _mapreduce)
for reducer in _reducers
    dict = RuleDSL.getdata(gs6, reducer)[]
    for (k, v) in dict
      println("$k => $(first(v, 10))")
    end
end

These previous examples are relatively straight forward in their logic. However, the
`mapper` and `reducer` rules can encapsulate complicated logic, both of them can be an
entire graph of great complexity. For example, the mapper can be the training and validation
of an entire ML model, and the reducer can be a bagging algorithm that joins multiple models
trained on different batches of data. We will show an example of a more complex use case in
the next tutorial.

## 4. Advantages of Julius Graph

### 4.1 Graph Composition vs Function Composition

You may find the high level rules in `RuleDSL` have a lot similarity to high order functions
in functional languages like Haskell, where a function can take another function as parameter.
So what are the main benefits of high order rule over the high order functions in a functional
language?

The key difference is that high level rules are for composing graphs, while high level
functions are for composing functions. The graph composition has a number of advantages over
function composition:

1. It does not create deep call stacks, the results of graph composition is nothing but
   another graph. Therefore, it is much easier for a developer to visualize and debug. With
   function composition, one has to use a debugger to access the intermediate results and
   call sequences, deep among the call stack of a program's runtime.
2. The resulting graph composition can be automatically distributed without code change. A
   generic graph distributor can analyze any graph and distribute it effectively to multiple
   worker computers. In contrast, the traditional functional code is permeated with loops
   and branches, making their runtime behavior unpredictable, thus cannot be distributed
   automatically and efficiently.
3. The graph composition is much more flexible. Once the graph is constructed, it can run in
   different mode. For example, the same graph can support both batch and streaming use
   cases without code changes, which is not possible in traditional functional programming.
4. Lastly, graph composition can mimic function composition, but the reverse is not true.
   The `mapreduce` rule is a good example how function composition can be replicated using
   graph composition. However, it is almost impossible to create the equivalent graph
   composition from function composition in traditional functional languages.

You have seen some of the benefits graph composition in this and previous tutorials. Next,
we will illustrate the 2nd benefit of automatically distributing the MapReduce pipeline to
multiple computers.

### 4.2 Distributed Map/Reduce

In order to show the automatic distribution, we set up a local cluster with 3 worker processes
managed by a master process running at a port of the local computer, which mimic a remote
master and worker processes running on multiple physical computers. Please note that the
local cluster automatically terminates after 15min of inactivity, so if you noticed the
local cluster no longer accessible after 15min, please re-run this entire tutorial notebook.

The following few lines of code starts the local cluster then connects to the master process,
through which we gain control to all the worker processes:

In [ ]:
using GraphEngine: RuleDSL, GraphVM

config = RuleDSL.newconfig(RuleDSL.Config(), :project => "MapReduce")
balancer = GraphVM.GlobalUnique()
my_domain = GraphVM.mydomain()

# draw a port number to start the local cluster esrvice
remoteport = GraphVM.drawdataport()

In [ ]:
# start a local master service at the given port

GraphVM.startlocalmasterservice(remoteport, 3)

gs = GraphVM.RemoteGraphProxy(config, my_domain => remoteport, balancer, GraphVM.GenericData())
GraphVM.wait4clusterinit(gs)

The following is the complete definition of the generic `mapreduce` rule and corresponding
functions for the word count example. Now we instantiate them in the remote cluster so that
we can run the distributed word count with distribution.

In [ ]:
GraphVM.@remote_eval gs begin
    using GraphEngine: RuleDSL, GraphVM
    using DataScience: ApplyFn
    using AtomExt, GraphIO
end

# wait for the server to complete the task before proceeding
GraphVM.waitcheckstatus(gs, RuleDSL.getconfig(config, :project));

GraphVM.@addrules gs mr begin
    echo(x::Any) = ApplyFn[identity, x]()

    mapper(batch::RuleDSL.NodeRef, mapfun::Function) = ApplyFn[mapfun](batch...)

    reducer(shuffled::RuleDSL.NodeRef, reducefun::Function) = ApplyFn[reducefun](shuffled...)

    splitbykey(mapper::RuleDSL.NodeRef, keyfunc::Function) = ApplyFn[splitbykey, keyfunc](mapper...)

    selectkey(dict::RuleDSL.NodeRef, key::Any; label="selectby $(key)") = ApplyFn[dict -> get(dict, key, [])](dict...)

    mergebykey(vecs::Vector{RuleDSL.NodeRef}) = ApplyFn[vcat](vecs...)

    shuffler(mappers::Vector{RuleDSL.NodeRef}, keyfunc::Function, keys::Set) = begin
        merged = Vector{NodeRef}()
        splits = RuleDSL.@ref(splitbykey(mapper, keyfunc) for mapper in mappers)
        for key in keys
            values = RuleDSL.@ref(selectkey(s, key) for s in splits)
            mergebykey = RuleDSL.@ref mergebykey(values; label="mergeby $key")
            push!(merged, mergebykey)
        end
        Alias(merged...)
    end

    mapreduce(
        batches::Vector{RuleDSL.NodeRef},
        mapper::RuleDSL.NodeRef,
        shuffler::RuleDSL.NodeRef,
        reducer::RuleDSL.NodeRef
    ) = begin
        mappers = [prepend(mapper, batch) for batch in batches]
        shuffler = prepend(shuffler, mappers)
        shuffled = RuleDSL.calcdeps(RuleDSL.@config, shuffler)
        reducers = [prepend(reducer, m) for m in shuffled]
        ApplyFn[merge](reducers...)
    end

    splitbykey(mapper::RuleDSL.NodeRef, keyfunc::Function, N::Int) = begin
        ApplyFn[splitbykey, x -> Int(hash(keyfunc(x)) % N)](mapper...)
    end

    shuffler(mappers::Vector{RuleDSL.NodeRef}, keyfunc::Function, N::Int) = begin
        splits = RuleDSL.@ref(splitbykey(mapper, keyfunc, N) for mapper in mappers)
        shuffled = Vector{NodeRef}()
        for key in 0:N-1
            selected = RuleDSL.@ref(selectkey(s, key) for s in splits)
            merged = RuleDSL.@ref mergebykey(selected; label="mergeby $key")
            push!(shuffled, merged)
        end
        Alias(shuffled...)
    end
end

GraphVM.@remote_eval gs begin
    import GraphEngine.RuleDSL: fwddata!

    prepend(ref::RuleDSL.NodeRef, firstarg::Any) = RuleDSL.NodeRef(ref.ns, ref.name, (firstarg, ref.params...), ref.meta)

    function splitbykey(keyfunc::Function, xs::Any)
        splits = Dict()
        for x in xs
            key = keyfunc(x)
            splits[key] = push!(get(splits, key, []), x)
        end
        return splits
    end

    wordmap(words::String) = [k => 1 for k in split(words)]

    function wordreduce(xs::Vector)
        count = Dict()
        for (key, _) in xs
            count[key] = get(count, key, 0) + 1
        end
        return count
    end

    import GraphEngine.RuleDSL: nodelabel
    function nodelabel(gs::RuleDSL.AbstractGraphState, ref::RuleDSL.NodeRef)
        shortrepr(x::Vector; sep=", ") = "["*join(shortrepr.(x), sep)*"]"
        shortrepr(p::Pair; sep="=>") = shortrepr(p.first) * sep * shortrepr(p.second)
        shortrepr(p::Dict; sep=", ") = "{" * join(shortrepr.(collect(p)), sep) * "}"
        shortrepr(x::Any; sep="") = repr(x)

        label = haskey(ref.meta, :label) ? ref.meta[:label] : "$(ref.ns).$(ref.name)"

        try
            data = RuleDSL.getdata(gs, ref)
            if isone(length(data))
                data = first(data)
            end
            label *= "\n" * shortrepr(data; sep = "\n")
        catch
            label *= ": n/a"
        end

        return label
    end
end

As you can see, there is no change in the RuleDSL and Julia functions at all, we simply sent
them to the remote cluster to instantiate. Afterwards, we can execute the MapReduce pipeline
with distribution. The distribution API, `RuleDSL.jobdeps` and `GraphVM.dispatchjobs!` are
explained in more detail in the next tutorial on Distributed ML pipeline, we won't
repeat them here.

In [ ]:
_sentences = ("Deer Bear River", "Car Car River", "Deer Car Bear")
_batches   = RuleDSL.@ref(mr.echo(s) for s in _sentences)

N = 3
_mapreduce = RuleDSL.@ref mr.mapreduce(
    _batches,
    RuleDSL.@ref(mr.mapper(Main.wordmap)),
    RuleDSL.@ref(mr.shuffler(first, N)),
    RuleDSL.@ref(mr.reducer(Main.wordreduce))
)

alljobs, ds = RuleDSL.jobdeps(config, [_mapreduce], Set([:splitbykey, :reducer]));

GraphVM.waitcheckstatus(gs, RuleDSL.getconfig(config, :project));
GraphVM.initgraph!(gs)
GraphVM.dispatchjobs!(gs, alljobs, 1);

In [ ]:
GraphVM.waitcheckstatus(gs, RuleDSL.getconfig(config, :project));
svg = GraphIO.postremotegraph(gs, port);

display("image/svg+xml", svg)

The resulting graph use different colors to highlight the placement of nodes to individual
remote workers. Nodes with the same color are placed and computed on the same worker
computer.

Upon close examination, we observe that the resulting graph distribution is optimal in
that the work load is evenly distributed amongst 3 workers, and the only shipment of data
happens during the shuffling stage, and the collation of final reducer results, when an
arrow connects two nodes with different colors. There is no unnecessary data transfer at all
in the resulting graph distribution.

The ability to automatically and optimally distribute graph without code change is a
powerful feature. Julius can handle the distribute of graphs as large as
hundreds millions nodes across hundreds of computers. Using Julius, the same code run
efficiently on one worker instance or hundreds of worker instances, without the need for any
manual tweaking and optimizations. Auto-scaling allows developers to quickly build and test
their rules and functions on the local computer, then immediately scale it up to run large
jobs and heavy workload in parallel without the need for any code changes. Julius'
autoscaling automates away one of the most time-consuming and expensive aspect in enterprise
systems, which is the constant needs for manual tweaking and optimization of the system for
better performance and scalability.

In a next toturial Distributed ML pipeline, we will dive into Julius distribution
and auto-scaling capability in much more depth, and compare to existing tools like Dask and
Dagger.jl.

## 5. Conclusion

The result speaks for itself: we built a generic MapReduce pipeline from scratch using 10
rules in RuleDSL and 20 lines of additional Julia code. The resulting MapReduce pipeline
implementation is generic, transparent and auto-scaling. Every intermediate calculation
result is fully visible to the user in our web UI, it automatically distributes to multiple
computers without any code changes for extreme scalability and performance.

Intrigued? If you are a developer, you should. To hear more about Julius Graph Engine,
contact us at `info@juliustech.co`, or go to our [website](http://juliustech.co) to schedule
a demo or sign up for free access for developers.

## Appendix: Additional Technical Tips & Notes

Here we explain some additional technical tips and points. We refer to the general structure
of a rule in RuleDSL as:

```julia
rulename(ruleargs::Any...) = Atom[atomargs...](dependentrule(depargs...))
```

* The `ApplyFn` atom is used extensively in this tutorial. Though it is convenient, it was
  only intended for simple analytical logic. For complex analytical algorithms, it is better
  to define individual Atoms for reusability and readability.
* There is an important difference between the `ruleargs` and `atomargs` in the rule syntax.
  The `ruleargs` is serialized and sent to remote worker during distribution, while atomargs
  are only created and used locally by individual workers. Therefore to enable distribution,
  every `ruleargs` has to be serializable with a stable hash, i.e. same object shall
  retrieve the same hash regardless which worker runs it. This requirement does not apply to
  atomargs. Julius use a customized hash function `RuleDSL.hashparam` for rule parameters,
  that support stable hashes for a wide variety of object types. However, the following are
  some instances where the serialization can fail or the hash becomes unstable:
  * If a rule parameter is a function closure with reference to any local variable, the
    serialization will fail.  A workaround is to move the function closure inside the body
    of the rule. For example, the first hash key shuffler will fail:
  ```julia
  # fail to serialize: local variable used in function closure
  N = 3
  _shuffler = RuleDSL.@ref mr.shuffler(x -> Int(hash(first(x)) % N), Set(collect(0:N-1)))
  ```
  but the second version of the shuffler will work fine:
  ```julia
  # closure moved inside the rule declaration
  _shuffler = RuleDSL.@ref mr.shuffle(first, N)
  ```
  * A complex struct is more likely to have unstable hashes, you can either make it inherit
    from `RuleDSL.ValueType` thus using the more stable `RuleDSL.hashparam`, or you can
    provide your own stable hash function by overriding the `RuleDSL.hashparam` method for
    the type in question. To help detect the potential serialization and hash stability
    issues in rules, we provide a convenient macro `RuleDSL.@isdistributable`, which will
    flag any node in a graph that cannot be safely distributed.
* You may be tempted to define a `mapreduce` rule that takes a mapper function and a reducer
  function, and create the `RuleDSL.@ref mr.mapper(func)` and `RuleDSL.@ref
  mr.reducer(func)` inside the `mapreduce` rule. As discussed before, this is less generic
  as the `mapper` and `reducer` rule shall not be restricted to simple wrappers like
  `RuleDSL.@ref mr.mapper(func)`. Instead, any rule can be used as the `mapper` and
  `reducer`, for example it could  represent a complex graph with sophisticated logic. As a
  matter of fact, they could very well be high order rules themselves.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*